In [ ]:
# 모든 파일 리스팅
import pandas as pd
from glob import glob
from pathlib import Path
from tqdm import tqdm
import random

train_imgs = glob('train_imgs/*.png')
test_imgs = glob('val_imgs/*.png')
len(train_imgs), len(test_imgs)